In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/titanictest/Titanic.csv')

In [ ]:
df

Значение колонок:
1. PassengerId - уникальный идентификатор пассажира
2. Survived - целевой признак (0 - не выжил 1 - выжил)
3. Pclass - класс билета (1, 2 или 3)
4. Name - имя и титул пассажира
5. Sex - пол пассажира (male или female)
6. Age - возсраст пассажира
7. SibSp - количество родственников и жены/мужы на борту
8. Parch - количество родителей и детей на борту
9. Ticket - номер билета
10. Fare - стоимость билета
11. Cabin - номерю каюты (если доступна)
12. Embarked - порт отправления (C, Q, S)

Выдвинутые гипотезы:
* выживание зависит от номера каюты
* выживание зависит от класса билета
* выживание зависит от возсраста
* выживание зависит от пола
* выживание зависит от id
* выживание зависит от SibSp
* выживание зависит от Parch

In [ ]:
df['Survived'].value_counts()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

Минимальные критерии для запуска модели машинного обучения
1) отсутствие пропусков
2) убрать не числовые типы

In [ ]:
df.info()

In [ ]:
кодируем данные 
колонки у которых тип данных object

In [ ]:
categorical_cols = df.select_dtypes(include=['object']).columns
label_encoders = {}

In [ ]:
categorical_cols

In [ ]:
from sklearn.preprocessing import LabelEncoder
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [ ]:
df

Видим что все колонны стали числами

In [ ]:
df.info()

теперь уберем пропуски в данных

In [ ]:
df.isnull().sum()

У нас пропуски только в колонке возраст
Заполним в ней нули медианным значением по этой же колонке

In [ ]:
df['Age'].fillna(df['Age'].median(), inplace=True)

In [ ]:
df.isnull().sum()

Сначала надо закодировать все в числа, затем заполнять пропуски

In [ ]:
X = df.drop(columns=['Survived'])
y = df['Survived']

In [ ]:
X

In [ ]:
df['Survived'].value_counts()

Создаем модель

* изменил test_size на 0.1

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create the merual network model
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.2f}')

In [ ]:
df.info()

график

In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()